In [1]:
from env import config
import requests

In [2]:
MINDSDB_UN=config("MINDSDB_UN", default=None)# martyna@mindsdb.com
MINDSDB_PW=config("MINDSDB_PW", default=None) # xxxxxxxx
assert MINDSDB_UN is not None
assert MINDSDB_PW is not None


MINDSDB_BASE_URL = "https://cloud.mindsdb.com/api"

In [3]:
FLIGHT_PRICE_PREDICTOR_MODEL = config("FLIGHT_PRICE_PREDICTOR_MODEL", default='flight_price_predictor')
to_train = "flight_price_predictor_a"
if FLIGHT_PRICE_PREDICTOR_MODEL == to_train:
    to_train = "flight_price_predictor_b"

In [4]:
print("Training", to_train)

Training flight_price_predictor_b


In [5]:
sql_command_1 =f"""DROP MODEL {to_train};"""

sql_command_2 = f"""
CREATE MODEL mindsdb.{to_train}
FROM ai_travel_agent (
  SELECT * FROM flight_prices
  )
PREDICT totalFare
ORDER BY flightDate
GROUP BY startingAirport, destinationAirport, isBasicEconomy, isRefundable, isNonStop, segmentsAirlineName
WINDOW 30
HORIZON 10;
"""

commands_to_run = [sql_command_1, sql_command_2]

In [6]:
def get_mindsdb_session():
    session = requests.Session()
    session.post('https://cloud.mindsdb.com/cloud/login', json={
        'email': MINDSDB_UN,
        'password': MINDSDB_PW
    })
    return session

In [7]:
def mindsdb_query(session, sql_query):
    endpoint = "/sql/query"
    url = f"{MINDSDB_BASE_URL}{endpoint}"
    headers = {"Content-Type": "application/json"}
    return session.post(url, json={"query": sql_query}, headers=headers)

In [8]:
session = get_mindsdb_session()
for cmd in commands_to_run:
    mindsdb_query(session, cmd)
session.close()

In [10]:
sql_command_3 = f"DESCRIBE PREDICTOR mindsdb.{to_train};"
session = get_mindsdb_session()
response = mindsdb_query(session, sql_command_3)
session.close()

In [20]:
response_data = response.json()
column_names = response_data['column_names']
# print(column_names)
dataset = response_data['data'][0]
# print(dataset)
for i, col in enumerate(column_names):
    print(col, dataset[i])

tables ['info', 'features', 'model', 'jsonai']
NAME flight_price_predictor_b
ENGINE lightwood
PROJECT mindsdb
ACTIVE True
VERSION 1
STATUS training
ACCURACY None
PREDICT totalFare
UPDATE_STATUS up_to_date
MINDSDB_VERSION 23.9.3.0
ERROR None
SELECT_DATA_QUERY SELECT * FROM flight_prices
TRAINING_OPTIONS {'target': 'totalFare', 'timeseries_settings': {'is_timeseries': True, 'order_by': 'flightDate', 'horizon': 10, 'window': 30, 'group_by': ['startingAirport', 'destinationAirport', 'isBasicEconomy', 'isRefundable', 'isNonStop', 'segmentsAirlineName']}}
TAG None
